In [1]:
!pip install dash pandas plotly
!pip install dash-bootstrap-components

In [2]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
import requests
from datetime import datetime
import random
import plotly.graph_objs as go
import pandas as pd

In [3]:
# Constants
API_KEY = 'Use_Your_Own_API_Key'
CITIES = {
    "Cape Town": (-33.9249, 18.4241),
    "Johannesburg": (-26.2041, 28.0473),
    "Durban": (-29.8587, 31.0218),
    "Polokwane": (-23.9045, 29.4689),
    "Mthatha": (-31.5897, 28.7844)
}

temperature_data = {city: [] for city in CITIES}
water_quality_data = {city: [] for city in CITIES}
air_quality_data = {city: [] for city in CITIES}
forecast_data = {city: [] for city in CITIES}

In [4]:
def get_weather_data(city, lat, lon):
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
        res = requests.get(url).json()
        return {
            'city': city,
            'temp': res['main']['temp'],
            'humidity': res['main']['humidity'],
            'weather': res['weather'][0]['description'],
            'alert': res['main']['temp'] > 35 or res['main']['humidity'] > 80
        }
    except:
        return {'city': city, 'temp': 'N/A', 'humidity': 'N/A', 'weather': 'Error', 'alert': False}


def get_forecast_data(city, lat, lon):
    try:
        url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
        res = requests.get(url).json()

        forecasts = []
        for item in res['list'][:8]:
            forecast_time = datetime.fromtimestamp(item['dt']).strftime('%H:%M %d/%m')
            forecasts.append({
                'time': forecast_time,
                'temp': item['main']['temp'],
                'humidity': item['main']['humidity'],
                'weather': item['weather'][0]['description'],
                'icon': item['weather'][0]['icon']
            })
        return forecasts
    except Exception as e:
        print(f"Error getting forecast: {e}")
        return []

def get_pm25_data(lat, lon):
    try:
        url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}"
        res = requests.get(url).json()
        return res['list'][0]['components']['pm2_5']
    except:
        return None

def simulate_water_quality(city):
    return {
        'pH': round(random.uniform(6.5, 8.5), 2),
        'dissolved_oxygen': round(random.uniform(4, 12), 2),
        'turbidity': round(random.uniform(0, 5), 2),
        'chlorine': round(random.uniform(0, 2), 2)
    }






def check_for_alerts(weather_data, water_data, air_data):
    alerts = []

    # Weather alerts
    for city, data in weather_data.items():
        if data.get('temp', 0) > 35:
            alerts.append(f"🌡️ Extreme heat in {city} ({data['temp']}°C)")
        if data.get('humidity', 0) > 80:
            alerts.append(f"💧 High humidity in {city} ({data['humidity']}%)")

    # Air quality alerts
    for city, data in air_data.items():
        pm25 = data.get('pm25', 0)
        if pm25 and pm25 > 35:
            alerts.append(f"🌬️ Poor air quality in {city} (PM2.5: {pm25} µg/m³)")

    # Water quality alerts
    for city, data in water_data.items():
        if data.get('pH', 7) < 6.5 or data.get('pH', 7) > 8.5:
            alerts.append(f"🚰 Unusual pH level in {city} water ({data['pH']})")
        if data.get('dissolved_oxygen', 0) < 5:
            alerts.append(f"🐟 Low dissolved oxygen in {city} water ({data['dissolved_oxygen']} mg/L)")

    return alerts

def create_weather_card(data):
    color = 'danger' if data['alert'] else 'success'
    return dbc.Card([
        dbc.CardHeader(html.H5(data['city'])),
        dbc.CardBody([
            html.P(f"🌡 Temp: {data['temp']}°C"),
            html.P(f"💧 Humidity: {data['humidity']}%"),
            html.P(f"🌥 Weather: {data['weather']}"),
            html.P(f"🌬 PM2.5: {data.get('pm25', 'N/A')} µg/m³"),
            html.P("⚠ Alert: Extreme Weather!", style={"color": "red"}) if data['alert'] else None
        ])
    ], color=color, outline=True, className="mb-3")

def create_temp_graph(city, data):
    return dcc.Graph(
        figure={
            'data': [go.Scatter(x=[d['time'] for d in data], y=[d['temp'] for d in data], mode='lines+markers')],
            'layout': go.Layout(title=f"{city} Temperature Trend", xaxis_title="Time", yaxis_title="°C")
        },
        className="mb-4"
    )

def create_forecast_graph(city, forecasts):
    if not forecasts:
        return html.Div("No forecast data available", className="text-muted")

    temp_trace = go.Scatter(
        x=[f['time'] for f in forecasts],
        y=[f['temp'] for f in forecasts],
        mode='lines+markers',
        name='Temperature (°C)',
        line=dict(color='red'))

    humidity_trace = go.Scatter(
        x=[f['time'] for f in forecasts],
        y=[f['humidity'] for f in forecasts],
        mode='lines+markers',
        name='Humidity (%)',
        yaxis='y2',
        line=dict(color='blue'))

    annotations = []
    for i, forecast in enumerate(forecasts):
        annotations.append(dict(
            x=forecast['time'],
            y=forecast['temp'] + 2,
            text=f"☁️" if 'cloud' in forecast['weather'].lower() else f"🌧️" if 'rain' in forecast['weather'].lower() else f"☀️",
            showarrow=False,
            xanchor='center',
            font=dict(size=15)
        ))

    fig = go.Figure(data=[temp_trace, humidity_trace])

    fig.update_layout(
        title=f"{city} 24-Hour Forecast",
        xaxis_title="Time",
        yaxis=dict(title='Temperature (°C)', color='red'),
        yaxis2=dict(title='Humidity (%)', color='blue', overlaying='y', side='right'),
        annotations=annotations,
        hovermode="x unified"
    )

    return dcc.Graph(figure=fig, className="mb-4")

def create_forecast_cards(forecasts):
    if not forecasts:
        return html.Div("No forecast data available", className="text-muted")

    cards = []
    for forecast in forecasts[:4]:
        weather_icon = {
            '01': '☀️',
            '02': '⛅',
            '03': '☁️',
            '04': '☁️',
            '09': '🌧️',
            '10': '🌦️',
            '11': '⛈️',
            '13': '❄️',
            '50': '🌫️'
        }.get(forecast['icon'][:2], '🌤️')

        cards.append(dbc.Col(
            dbc.Card([
                dbc.CardHeader(html.H5(forecast['time'])),
                dbc.CardBody([
                    html.H3(weather_icon, className="text-center mb-2"),
                    html.P(f"🌡 {forecast['temp']}°C"),
                    html.P(f"💧 {forecast['humidity']}%"),
                    html.P(f"{forecast['weather']}")
                ])
            ], className="h-100"),
            md=3
        ))

    return dbc.Row(cards, className="g-4")

def create_water_card(city_data):
    return dbc.Card([
        dbc.CardHeader(html.H5(f"{city_data['city']} Water Quality")),
        dbc.CardBody([
            html.P(f"💧 pH: {city_data['pH']}"),
            html.P(f"🌊 Dissolved Oxygen: {city_data['dissolved_oxygen']} mg/L"),
            html.P(f"🌫 Turbidity: {city_data['turbidity']} NTU"),
            html.P(f"🚰 Chlorine: {city_data['chlorine']} mg/L")
        ])
    ], color="info", outline=True, className="mb-3")

def create_air_card(city, pm25):
    return dbc.Card([
        dbc.CardHeader(html.H5(f"{city} Air Quality")),
        dbc.CardBody([
            html.P(f"🌬 PM2.5: {pm25} µg/m³"),
            html.P("⚠ Unhealthy Air!" if pm25 and pm25 > 35 else "Air quality is acceptable.")
        ])
    ], color="secondary", outline=True, className="mb-3")

# Initialize App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])
app.title = "SA Environment Monitor"

navbar = dbc.NavbarSimple(
    brand="🌍 SA Environmental Dashboard",
    color="dark",
    dark=True,
    children=[
        dbc.NavItem(dbc.NavLink("Climate", href="#", id="nav-climate")),
        dbc.NavItem(dbc.NavLink("Water Quality", href="#", id="nav-water")),
        dbc.NavItem(dbc.NavLink("Air Quality", href="#", id="nav-air")),
        dbc.Badge(id="alert-badge", color="danger", pill=True, className="ms-2"),
    ]
)

app.layout = dbc.Container([
    navbar,

    # Alert toast
    dbc.Toast(
        id="alert-toast",
        header="Alert!",
        is_open=False,
        dismissable=True,
        icon="danger",
        style={"position": "fixed", "top": 66, "right": 10, "width": 350, "zIndex": 1000},
    ),

    # Optional sound alert
    html.Audio(id='alert-sound', src='https://assets.mixkit.co/sfx/preview/mixkit-alarm-digital-clock-beep-989.mp3', autoPlay=False),

    html.Div([
        html.H5("⏰ Current Time: ", style={"display": "inline-block"}),
        html.Span(id='live-time', style={"fontWeight": "bold"})
    ], className="my-3"),

    dcc.Tabs(id='tabs', value='climate', children=[
        dcc.Tab(label='🌦 Climate', value='climate', children=[
            dbc.Row(id='weather-cards', className="g-4"),
            html.Div(id='temp-graphs', className="bg-dark p-3 rounded"),
            html.Hr(),
            html.H4("🌤️ Weather Forecast", className="mt-4"),
            dcc.Dropdown(
                id='forecast-city-selector',
                options=[{'label': city, 'value': city} for city in CITIES.keys()],
                value=list(CITIES.keys())[0],
                clearable=False,
                className="mb-3"
            ),
            html.Div(id='forecast-graph'),
            html.Div(id='forecast-cards')
        ]),
        dcc.Tab(label='🚰 Water Quality', value='water', children=[
            dbc.Row(id='water-cards', className="g-4 bg-info bg-opacity-25 p-3 rounded")
        ]),
        dcc.Tab(label='🌬 Air Quality', value='air', children=[
            dbc.Row(id='air-cards', className="g-4 bg-secondary bg-opacity-25 p-3 rounded")
        ])
    ]),

    html.Div([
        html.Button("⬇ Download Temperature Report", id="download-temp-btn", className="btn btn-primary m-2"),
        html.Button("⬇ Download Water Report", id="download-water-btn", className="btn btn-info m-2"),
        html.Button("⬇ Download Air Report", id="download-air-btn", className="btn btn-secondary m-2"),
        dcc.Download(id="download-temp"),
        dcc.Download(id="download-water"),
        dcc.Download(id="download-air")
    ]),

    dcc.Interval(id='refresh-interval', interval=60 * 1000, n_intervals=0),
    dcc.Interval(id='clock', interval=1000, n_intervals=0)
], fluid=True)

# Callbacks
@app.callback(Output('live-time', 'children'), Input('clock', 'n_intervals'))
def update_time(n):
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@app.callback(
    [Output('weather-cards', 'children'),
     Output('temp-graphs', 'children'),
     Output('water-cards', 'children'),
     Output('air-cards', 'children'),
     Output('alert-toast', 'is_open'),
     Output('alert-toast', 'children'),
     Output('alert-toast', 'header'),
     Output('alert-badge', 'children'),
     Output('alert-sound', 'autoPlay')],
    Input('refresh-interval', 'n_intervals')
)
def update_data(n):
    weather_cards, temp_graphs, water_cards, air_cards = [], [], [], []
    weather_data = {}
    water_data = {}
    air_data = {}

    for city, (lat, lon) in CITIES.items():
        time_now = datetime.now().strftime("%H:%M:%S")

        # Weather
        weather = get_weather_data(city, lat, lon)
        weather['pm25'] = get_pm25_data(lat, lon)
        weather_data[city] = weather
        temperature_data[city].append({'time': time_now, 'temp': weather['temp']})
        weather_cards.append(dbc.Col(create_weather_card(weather), md=4))
        temp_graphs.append(create_temp_graph(city, temperature_data[city]))

        # Water
        water = simulate_water_quality(city)
        water_data[city] = water
        water_quality_data[city].append({'time': time_now, **water})
        water_cards.append(dbc.Col(create_water_card({**water, 'city': city}), md=4))

        # Air
        air_quality = {'pm25': weather['pm25']}
        air_data[city] = air_quality
        air_quality_data[city].append({'time': time_now, **air_quality})
        air_cards.append(dbc.Col(create_air_card(city, weather['pm25']), md=4))

    # Check for alerts
    alerts = check_for_alerts(weather_data, water_data, air_data)
    show_alert = len(alerts) > 0
    alert_content = html.Ul([html.Li(alert) for alert in alerts])
    alert_header = f"⚠️ {len(alerts)} Alert{'s' if len(alerts) > 1 else ''} Detected"
    play_sound = show_alert

    return (weather_cards, temp_graphs, water_cards, air_cards,
            show_alert, alert_content, alert_header, len(alerts), play_sound)

@app.callback(
    [Output('forecast-graph', 'children'),
     Output('forecast-cards', 'children')],
    [Input('forecast-city-selector', 'value'),
     Input('refresh-interval', 'n_intervals')]
)
def update_forecast(city, n):
    lat, lon = CITIES[city]
    forecasts = get_forecast_data(city, lat, lon)
    forecast_data[city] = forecasts

    graph = create_forecast_graph(city, forecasts)
    cards = create_forecast_cards(forecasts)

    return graph, cards

@app.callback(
    Output("download-temp", "data"),
    Input("download-temp-btn", "n_clicks"),
    prevent_initial_call=True
)
def download_temp_report(n_clicks):
    records = []
    for city, entries in temperature_data.items():
        for entry in entries:
            records.append({'City': city, 'Time': entry['time'], 'Temperature (°C)': entry['temp']})
    df = pd.DataFrame(records)
    return dcc.send_data_frame(df.to_csv, filename="temperature_report.csv", index=False)

@app.callback(
    Output("download-water", "data"),
    Input("download-water-btn", "n_clicks"),
    prevent_initial_call=True
)
def download_water_report(n_clicks):
    records = []
    for city, entries in water_quality_data.items():
        for entry in entries:
            records.append({'City': city, 'Time': entry['time'], **{k: v for k, v in entry.items() if k != 'time'}})
    df = pd.DataFrame(records)
    return dcc.send_data_frame(df.to_csv, filename="water_quality_report.csv", index=False)

@app.callback(
    Output("download-air", "data"),
    Input("download-air-btn", "n_clicks"),
    prevent_initial_call=True
)
def download_air_report(n_clicks):
    records = []
    for city, entries in air_quality_data.items():
        for entry in entries:
            records.append({'City': city, 'Time': entry['time'], 'PM2.5 (µg/m³)': entry['pm25']})
    df = pd.DataFrame(records)
    return dcc.send_data_frame(df.to_csv, filename="air_quality_report.csv", index=False)

if __name__ == '__main__':
    app.run(debug=True)
